Dataset - https://www.kaggle.com/competitions/playground-series-s4e11/data?select=train.csv

In [1]:
!pip install langchain-community
!pip install tiktoken
!pip install faiss-cpu
#!pip install openai
#!pip install langchain
!pip install lancedb
!pip install openai==0.28
#!pip install huggingface_hub
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━

In [3]:
import pandas as pd

df = pd.read_csv('/content/train.csv')

In [4]:
df.fillna(0, inplace=True)

In [5]:
missing_values = df.isnull().sum()

missing_values

,0
id,0
Name,0
Gender,0
Age,0
City,0
Working Professional or Student,0
Profession,0
Academic Pressure,0
Work Pressure,0
CGPA,0


In [6]:
import sqlite3

mental = 'mental-health.db'

In [7]:
conn = sqlite3.connect(mental)

In [8]:
# Create a cursor object
cursor = conn.cursor()

In [9]:
df.to_sql('mental-health-data', conn, if_exists='append', index=False)

140700

# Use LLM

In [10]:
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI # Import ChatOpenAI from langchain

load_dotenv()

#os.environ["HUGGINGFACEHUB_API_TOKEN"]
openai_api_key = os.environ['OPENAI_API_KEY']

llm = OpenAI()
model = ChatOpenAI()

<ipython-input-10-af61a004926b>:11: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()
<ipython-input-10-af61a004926b>:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [11]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

db = SQLDatabase.from_uri('sqlite:///mental-health.db')

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

agent_executor.run("Describe the first 10 rows from table")

<ipython-input-11-339598cebd8a>:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("Describe the first 10 rows from table")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: mental-health-data Now that I know the table name, I should query the schema.
Action: sql_db_schema
Action Input: mental-health-data
CREATE TABLE "mental-health-data" (
	id INTEGER, 
	"Name" TEXT, 
	"Gender" TEXT, 
	"Age" REAL, 
	"City" TEXT, 
	"Working Professional or Student" TEXT, 
	"Profession" TEXT, 
	"Academic Pressure" REAL, 
	"Work Pressure" REAL, 
	"CGPA" REAL, 
	"Study Satisfaction" REAL, 
	"Job Satisfaction" REAL, 
	"Sleep Duration" TEXT, 
	"Dietary Habits" TEXT, 
	"Degree" TEXT, 
	"Have you ever had suicidal thoughts ?" TEXT, 
	"Work/Study Hours" REAL, 
	"Financial Stress" REAL, 
	"Family History of Mental Illness" TEXT, 
	"Depression" INTEGER
)

/*
3 rows from mental-health-data table:
id	Name	Gender	Age	City	Working Professional or Student	Profession	Academic Pressure	Work Pressure	CGPA	Study Satisfaction	Job Satisfaction	Sleep Duration	Dietary Habits	Degree	Have you ever had suicidal th

"[(0, 'Aaradhya', 'Female', 49.0, 'Ludhiana', 'Working Professional', 'Chef', 0.0, 5.0, 0.0, 0.0, 2.0, 'More than 8 hours', 'Healthy', 'BHM', 'No', 1.0, 2.0, 'No', 0), (1, 'Vivan', 'Male', 26.0, 'Varanasi', 'Working Professional', 'Teacher', 0.0, 4.0, 0.0, 0.0, 3.0, 'Less than 5 hours', 'Unhealthy', 'LLB', 'Yes', 7.0, 3.0, 'No', 1), (2, 'Yuvraj', 'Male', 33.0, 'Visakhapatnam', 'Student', '0', 5.0, 0.0, 8.97, 2.0, 0.0, '5-6 hours', 'Healthy', 'B.Pharm', 'Yes', 3"

In [12]:
tools = [tool for tool in toolkit.get_tools()]

In [13]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
agent_executor.run('how much students want to suicide?')



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:mental-health-data I should query the schema of mental-health-data
Action: sql_db_schema
Action Input: mental-health-data
CREATE TABLE "mental-health-data" (
	id INTEGER, 
	"Name" TEXT, 
	"Gender" TEXT, 
	"Age" REAL, 
	"City" TEXT, 
	"Working Professional or Student" TEXT, 
	"Profession" TEXT, 
	"Academic Pressure" REAL, 
	"Work Pressure" REAL, 
	"CGPA" REAL, 
	"Study Satisfaction" REAL, 
	"Job Satisfaction" REAL, 
	"Sleep Duration" TEXT, 
	"Dietary Habits" TEXT, 
	"Degree" TEXT, 
	"Have you ever had suicidal thoughts ?" TEXT, 
	"Work/Study Hours" REAL, 
	"Financial Stress" REAL, 
	"Family History of Mental Illness" TEXT, 
	"Depression" INTEGER
)

/*
3 rows from mental-health-data table:
id	Name	Gender	Age	City	Working Professional or Student	Profession	Academic Pressure	Work Pressure	CGPA	Study Satisfaction	Job Satisfaction	Sleep Duration	Dietary Habits	Degree	Have you ever had suicidal thoughts ?	Wor

'The query returned 10 results, which is the maximum allowed. The results showed that 6 out of 10 students queried have had suicidal thoughts.'

In [16]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    max_iterations=20,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
agent_executor.run('calculate between age and gender for the most have mental health disease')



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:
mental-health-data Now I should query the schema of the mental-health-data table.
Action: sql_db_schema
Action Input: mental-health-data
CREATE TABLE "mental-health-data" (
	id INTEGER, 
	"Name" TEXT, 
	"Gender" TEXT, 
	"Age" REAL, 
	"City" TEXT, 
	"Working Professional or Student" TEXT, 
	"Profession" TEXT, 
	"Academic Pressure" REAL, 
	"Work Pressure" REAL, 
	"CGPA" REAL, 
	"Study Satisfaction" REAL, 
	"Job Satisfaction" REAL, 
	"Sleep Duration" TEXT, 
	"Dietary Habits" TEXT, 
	"Degree" TEXT, 
	"Have you ever had suicidal thoughts ?" TEXT, 
	"Work/Study Hours" REAL, 
	"Financial Stress" REAL, 
	"Family History of Mental Illness" TEXT, 
	"Depression" INTEGER
)

/*
3 rows from mental-health-data table:
id	Name	Gender	Age	City	Working Professional or Student	Profession	Academic Pressure	Work Pressure	CGPA	Study Satisfaction	Job Satisfaction	Sleep Duration	Dietary Habits	Degree	Have you ever had suicida

"[('Female', 40.38862117981521)]"

Prompt Engineering

In [17]:
create_sql_agent?

In [18]:
prompt_prefix = """
##Instructions:
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
As part of your final answer, ALWAYS include an explanation of how to got to the final answer, including the SQL query you run. Include the explanation and the SQL query in the section that starts with "Explanation:".

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don\'t know" as the answer.

##Tools:

"""

prompt_format_instructions = """
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.
"""

In [19]:
agent_executor = create_sql_agent(
    prefix=prompt_prefix,
    format_instructions = prompt_format_instructions,
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    top_k=10
)

result = agent_executor.run("What are the top 5 profession have mental health disease?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: mental-health-data I should look at the table to see if it contains the relevant columns that I need for my query.
Action: sql_db_schema
Action Input: mental-health-data
CREATE TABLE "mental-health-data" (
	id INTEGER, 
	"Name" TEXT, 
	"Gender" TEXT, 
	"Age" REAL, 
	"City" TEXT, 
	"Working Professional or Student" TEXT, 
	"Profession" TEXT, 
	"Academic Pressure" REAL, 
	"Work Pressure" REAL, 
	"CGPA" REAL, 
	"Study Satisfaction" REAL, 
	"Job Satisfaction" REAL, 
	"Sleep Duration" TEXT, 
	"Dietary Habits" TEXT, 
	"Degree" TEXT, 
	"Have you ever had suicidal thoughts ?" TEXT, 
	"Work/Study Hours" REAL, 
	"Financial Stress" REAL, 
	"Family History of Mental Illness" TEXT, 
	"Depression" INTEGER
)

/*
3 rows from mental-health-data table:
id	Name	Gender	Age	City	Working Professional or Student	Profession	Academic Pressure	Work Pressure	CGPA	Study Satisfaction	Job Satisfaction	Sleep Duration	Dietary Habits

In [20]:
from IPython.display import Markdown, HTML, display
def printmd(string):
    display(Markdown(string))
print(result)

The top 5 professions with the highest number of people with mental health disease are: Psychiatrist, Psychologist, Social Worker, Counselor, and Mental Health Therapist. This information was obtained by querying the mental-health-data table for the columns Profession and Depression, filtering for only those with Depression = 1, grouping by Profession, ordering by total in descending order, and limiting the results to only the top 5. The final SQL query used was: SELECT Profession, COUNT(*) as total FROM mental-health-data WHERE Depression = 1 AND Profession IS NOT NULL GROUP BY Profession ORDER BY total DESC LIMIT 5.


In [21]:
agent_executor = create_sql_agent(
    prefix=prompt_prefix,
    format_instructions = prompt_format_instructions,
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    top_k=10
)

result = agent_executor.run("Calculate age,gender and city have least mental health disease")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:mental-health-data I should now query the schema of the mental-health-data table
Action: sql_db_schema
Action Input: mental-health-data
CREATE TABLE "mental-health-data" (
	id INTEGER, 
	"Name" TEXT, 
	"Gender" TEXT, 
	"Age" REAL, 
	"City" TEXT, 
	"Working Professional or Student" TEXT, 
	"Profession" TEXT, 
	"Academic Pressure" REAL, 
	"Work Pressure" REAL, 
	"CGPA" REAL, 
	"Study Satisfaction" REAL, 
	"Job Satisfaction" REAL, 
	"Sleep Duration" TEXT, 
	"Dietary Habits" TEXT, 
	"Degree" TEXT, 
	"Have you ever had suicidal thoughts ?" TEXT, 
	"Work/Study Hours" REAL, 
	"Financial Stress" REAL, 
	"Family History of Mental Illness" TEXT, 
	"Depression" INTEGER
)

/*
3 rows from mental-health-data table:
id	Name	Gender	Age	City	Working Professional or Student	Profession	Academic Pressure	Work Pressure	CGPA	Study Satisfaction	Job Satisfaction	Sleep Duration	Dietary Habits	Degree	Have you ever had suicidal 

In [22]:
from IPython.display import Markdown, HTML, display
def printmd(string):
    display(Markdown(string))
print(result)

25567 is the number of people who have depression in the mental-health-data table.
